# Hellings-Downs curve


The [Hellings-Downs curve](https://arxiv.org/abs/2208.07230) is used for the detection of the stochastic GW background.

What does _our_ HD-equivalent curve look like using the state-space model and frequencies?

---

### Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import json 
import pandas as pd 
try:
    sys.path.remove("../py_src") # Hacky way to add higher directory to python modules path. 
except:
    pass
sys.path.append("../py_src") # Means that I dont have to make src/ a proper python package

Now to start, lets make a function that creates some synthetic data for a particular strain, measurement noise and noise seed AND a function that gets a likelihood value given that data 

# What do the likelihood curves look like for $K$ sources? 

In [42]:

from system_parameters import SystemParameters
from pulsars import Pulsars
from synthetic_data import SyntheticData
from model import LinearModel
from kalman_filter import KalmanFilter
from priors import priors_dict,bilby_priors_dict
import numpy as np 

import sys
#Function to create synthetic data
def create_synthetic_data(h,measurement_noise,seed,K):


    P    = SystemParameters(h=h,σp=process_noise,σm=measurement_noise,seed=seed,num_gw_sources=K)       #define the system parameters as a class
    PTA  = Pulsars(P)               # setup the PTA
    data = SyntheticData(PTA,P)     # generate some synthetic data

    return P,PTA,data

#Fit model to data and return likelihood
def get_likelihood(data,parameter_name,parameter_value,parameter_index,P,PTA):

    _P = copy.copy(P)
    _PTA = copy.copy(PTA)

   
    #Define the model 
    model = LinearModel(_P)

    #Initialise the Kalman filter
    KF = KalmanFilter(model,data.f_measured,_PTA,P.num_gw_sources)

    #Get the
    guessed_parameters = priors_dict(_PTA,_P) #These are the true parameters
   


    print("the paramters dict is as follows")
    print(guessed_parameters)

    #sys.exit()
    sliced = guessed_parameters[parameter_name]
    sliced[parameter_index] = parameter_value

    guessed_parameters[parameter_name] = sliced
    
    #print("value after:",guessed_parameters[parameter_name] )

    
    model_likelihood = KF.likelihood(guessed_parameters)

    return model_likelihood

In [77]:

import copy 
import logging 

#Setup parameters for K sources and N pulsars
h = None    #set randomly
measurement_noise = 1e-11
process_noise = None #randomly assigned
K = 10
seed = 1237



#Initialise 
P    = SystemParameters(h=h,σp=process_noise,σm=measurement_noise,seed=seed,num_gw_sources=K)       #define the system parameters as a class
PTA  = Pulsars(P)               # setup the PTA
data = SyntheticData(PTA,P)     # generate some synthetic data


#Define the model 
model = LinearModel(P)

#Initialise the Kalman filter
KF = KalmanFilter(model,data.f_measured,PTA,P.num_gw_sources)

#Bilby
init_parameters_optimal, priors_optimal = bilby_priors_dict(PTA,P,set_parameters_as_known=True)
params = priors_optimal.sample(1)
optimal_model_likelihood = KF.likelihood(params)


init_parameters, priors = bilby_priors_dict(PTA,P,set_parameters_as_known=False)
params = priors.sample(1)
model_likelihood = KF.likelihood(params)



print(len(params))

print(optimal_model_likelihood,model_likelihood,optimal_model_likelihood-model_likelihood)


# print("the params returned by the bilby sampler are")
# for key,value in params.items():
#     print(key,value)

# import sys
# sys.exit()
#Run the KF with some randomly sampled params to check everything looks ok





INFO:root:Welcome to the Kalman Filter Nested Sampler for PTA GW systems


INFO:root:Random seed is 1237
INFO:root:Multiple GW sources requested. Overwriting default GW parameters and randomly sampling
INFO:root:You are assigning the σp terms randomly


Running with 10 GW sources 


INFO:root:You are including the PSR terms in your synthetic data generation
INFO:root:Generating synthetic data
INFO:root:You are using the Pulsar terms measurement model
INFO:root:Setting the bilby priors dict
INFO:root:Setting fully informative priors on GW parameters
INFO:root:Setting fully informative priors on PSR parameters
INFO:root:Setting the bilby priors dict
INFO:root:Setting uninformative priors on GW parameters
INFO:root:Setting uninformative priors on PSR parameters
INFO:root:Sigma p true value is sigma_p0 9.769327690311607e-20
INFO:root:Sigma p true value is sigma_p1 3.8639377766942156e-20
INFO:root:Sigma p true value is sigma_p2 9.240137714263159e-20
INFO:root:Sigma p true value is sigma_p3 2.6907549962490873e-20
INFO:root:Sigma p true value is sigma_p4 3.2590608783005556e-20
INFO:root:Sigma p true value is sigma_p5 1.2691032063697638e-20
INFO:root:Sigma p true value is sigma_p6 2.4934863032025724e-20
INFO:root:Sigma p true value is sigma_p7 3.2534858949440415e-20
INFO:

776
585736.316277382 583884.233049336 1852.083228045958


In [58]:
model_likelihood

585658.6185181594